Пирогов Артём. группа 6409. Лабораторная № 2

### Задание
Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. 

Получившийся отчёт сохранить в формате Apache Parquet.

In [3]:
from pyspark import SparkContext, SparkConf
from datetime import datetime
from pyspark.sql import SparkSession

from typing import NamedTuple
import pandas as pd
import os
import numpy as np


In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'

In [6]:
sc = SparkSession.builder.appName("Project2").master("local[*]").getOrCreate()
sc

In [42]:
posts_sample = sc.read.format('xml').option('rowTag', 'row').load("data/posts_sample.xml")

In [63]:
posts_sample.take(3)

[Row(_AcceptedAnswerId=7, _AnswerCount=13, _Body="<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n  <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code></p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n", _ClosedDate=None, _CommentCount=2, _CommunityOwnedDate=datetime.datetime(2012, 10, 31, 20, 42, 47, 213000), _CreationDate=datetime.datetime(2008, 8, 1, 2, 42, 52, 667000), _FavoriteCount=48, _Id=4, _LastActivityDate=datetime.datetime(2019, 7, 19, 5, 39, 54, 173000), _LastEditDate=datetime.datetime(2019, 7, 19, 5, 39, 54, 173000), _LastEditorDisplayName='Rich B', _LastEditorUserId=3641067, _OwnerDisplayName=None, _OwnerUser

In [162]:
posts_sample_period = posts_sample.filter(lambda row: (2009 < row._CreationDate.year) and (row._CreationDate.year < 2021))
posts_sample_period.take(10)

[Row(_AcceptedAnswerId=None, _AnswerCount=None, _Body='<p>No. (And more stuff to round this up to 15 characters...)</p>\n', _ClosedDate=None, _CommentCount=6, _CommunityOwnedDate=None, _CreationDate=datetime.datetime(2010, 9, 20, 20, 18, 14, 580000), _FavoriteCount=None, _Id=3753373, _LastActivityDate=datetime.datetime(2010, 9, 20, 20, 18, 14, 580000), _LastEditDate=None, _LastEditorDisplayName=None, _LastEditorUserId=None, _OwnerDisplayName=None, _OwnerUserId=10293, _ParentId=3753364, _PostTypeId=2, _Score=13, _Tags=None, _Title=None, _ViewCount=None),
 Row(_AcceptedAnswerId=None, _AnswerCount=None, _Body='<p>I am not aware of a way to use the URL parameters the way you have indicated. If anyone knows better, do correct me.</p>\n\n<p>I faced a similar situation some time ago and made do with a thin wrapper over the <code>list_detail</code> view. </p>\n\n<pre><code># views.py\nfrom django.views.generic.list_detail import object_list\n\ndef object_list_wrapper(*args, **kwargs):\n    cat

In [223]:
languages_name = sc.read.csv("data/programming-languages.csv")

In [224]:
languages_name.take(10)

[Row(_c0='name', _c1='wikipedia_url'),
 Row(_c0='A# .NET', _c1='https://en.wikipedia.org/wiki/A_Sharp_(.NET)'),
 Row(_c0='A# (Axiom)', _c1='https://en.wikipedia.org/wiki/A_Sharp_(Axiom)'),
 Row(_c0='A-0 System', _c1='https://en.wikipedia.org/wiki/A-0_System'),
 Row(_c0='A+', _c1='https://en.wikipedia.org/wiki/A%2B_(programming_language)'),
 Row(_c0='A++', _c1='https://en.wikipedia.org/wiki/A%2B%2B'),
 Row(_c0='ABAP', _c1='https://en.wikipedia.org/wiki/ABAP'),
 Row(_c0='ABC', _c1='https://en.wikipedia.org/wiki/ABC_(programming_language)'),
 Row(_c0='ABC ALGOL', _c1='https://en.wikipedia.org/wiki/ABC_ALGOL'),
 Row(_c0='ABSET', _c1='https://en.wikipedia.org/wiki/ABSET')]

In [225]:
languages_name_header = languages_name.first()
languages_name_header

Row(_c0='name', _c1='wikipedia_url')

In [226]:
languages_name

DataFrame[_c0: string, _c1: string]

In [227]:
names = []
for x in languages_name.collect():
    names.append(str(x[0]))

names[0:20]

['name',
 'A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS',
 'ACC',
 'Accent',
 'Ace DASL',
 'ACL2',
 'ACT-III',
 'Action!',
 'ActionScript',
 'Ada',
 'Adenine']

In [228]:
count_langueges = len(names)-1
count_langueges

700

In [229]:
def includes_name(x):
    tag = None
    for name in names:
        n = '<' + name.lower() + '>'
        if n in str(x._Tags).lower():
            tag = name
            break
    if tag is None:
        tag = 'No'
            
    return (x[6], tag)

In [230]:
period_language = posts_sample_period.map(includes_name).filter(lambda x: x[1] != 'No')
period_language.take(5)

[(datetime.datetime(2010, 9, 23, 16, 13, 59, 443000), 'Java'),
 (datetime.datetime(2010, 9, 26, 21, 7, 4, 840000), 'PHP'),
 (datetime.datetime(2010, 9, 30, 22, 27, 56, 320000), 'Ruby'),
 (datetime.datetime(2010, 10, 1, 15, 52, 42, 210000), 'C'),
 (datetime.datetime(2010, 10, 5, 1, 5, 50, 150000), 'PHP')]

In [232]:
period_language_group = period_language.keyBy(lambda row: (row[0].year,row[1])).aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2).sortBy(lambda x: x[1], ascending=False)

In [233]:
period_language_group.take(5)

[((2016, 'JavaScript'), 272),
 ((2015, 'JavaScript'), 270),
 ((2017, 'JavaScript'), 244),
 ((2014, 'JavaScript'), 235),
 ((2014, 'Java'), 228)]

In [234]:
list_period_language_group = []

for row in period_language_group.collect():
    list_row = []
    for i in row[0]:
        list_row.append(i)
    list_row.append(row[1])
    
    list_period_language_group.append([list_row[0], list_row[1], list_row[2]])
    

In [235]:
list_period_language_group[0:10]

[[2016, 'JavaScript', 272],
 [2015, 'JavaScript', 270],
 [2017, 'JavaScript', 244],
 [2014, 'JavaScript', 235],
 [2014, 'Java', 228],
 [2018, 'Python', 214],
 [2015, 'Java', 208],
 [2017, 'Java', 204],
 [2013, 'JavaScript', 196],
 [2018, 'JavaScript', 196]]

In [236]:
from pyspark.sql import Row
rows = Row('Year','Language','Count')
df=sc.createDataFrame([rows(x,y,z) for x,y,z in (list_period_language_group[0:10])])

In [237]:
df.show()

+----+----------+-----+
|Year|  Language|Count|
+----+----------+-----+
|2016|JavaScript|  272|
|2015|JavaScript|  270|
|2017|JavaScript|  244|
|2014|JavaScript|  235|
|2014|      Java|  228|
|2018|    Python|  214|
|2015|      Java|  208|
|2017|      Java|  204|
|2013|JavaScript|  196|
|2018|JavaScript|  196|
+----+----------+-----+


In [239]:
# Получившийся отчёт сохранить в формате Apache Parquet
df.write.parquet("10_language.parquet")